In [3]:
#data manipulation
import math
import pandas as pd
import numpy as np

#Pytorch geometric
import torch   
import torch_geometric
from torch import nn, Tensor
from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from torch.nn import Linear, BCELoss, LSTM, Dropout
import torch_geometric.transforms as T
import torch.nn.functional as F

from torch_geometric.nn import global_mean_pool as gap,  global_max_pool as gmp, global_add_pool as gsp

#rdkit
from rdkit import Chem                      
from rdkit.Chem import GetAdjacencyMatrix       
from scipy.sparse import coo_matrix
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs

#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#chemprop
import chemprop
from chemprop.args import TrainArgs, PredictArgs
from chemprop.train import cross_validate, run_training, make_predictions

#sklearn
import sklearn
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold

#shuffle
from random import shuffle

#GPU
import gc

## Utility Functions

### Chem

In [4]:
def onek_encoding_unk(value, choices):
    """
    Creates a one-hot encoding with an extra category for uncommon values.

    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is -1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding

In [5]:
class Featurization_parameters:
    """
    A class holding molecule featurization parameters as attributes.
    """
    def __init__(self) -> None:

        # Atom feature sizes
        self.MAX_ATOMIC_NUM = 100
        #for one-hot-encoding
        self.ATOM_FEATURES = {
            'atomic_num': list(range(self.MAX_ATOMIC_NUM)),
            'degree': [0, 1, 2, 3, 4, 5],
            'formal_charge': [-1, -2, 1, 2, 0],
            'chiral_tag': [0, 1, 2, 3],
            'num_Hs': [0, 1, 2, 3, 4],
            'hybridization': [
                Chem.rdchem.HybridizationType.SP,
                Chem.rdchem.HybridizationType.SP2,
                Chem.rdchem.HybridizationType.SP3,
                Chem.rdchem.HybridizationType.SP3D,
                Chem.rdchem.HybridizationType.SP3D2
            ],
        }

        # Distance feature sizes
        self.PATH_DISTANCE_BINS = list(range(10))
        self.THREE_D_DISTANCE_MAX = 20
        self.THREE_D_DISTANCE_STEP = 1
        self.THREE_D_DISTANCE_BINS = list(range(0, self.THREE_D_DISTANCE_MAX + 1, self.THREE_D_DISTANCE_STEP))

        # len(choices) + 1 to include room for uncommon values; + 2 at end for IsAromatic and mass
        self.ATOM_FDIM = sum(len(choices) + 1 for choices in self.ATOM_FEATURES.values()) + 2
        self.EXTRA_ATOM_FDIM = 0
        self.BOND_FDIM = 14
        self.EXTRA_BOND_FDIM = 0
        self.REACTION_MODE = None
        self.EXPLICIT_H = False
        self.REACTION = False

In [6]:
PARAMS = Featurization_parameters()

In [7]:
def atom_features(atom: Chem.rdchem.Atom, functional_groups=None):
    """
    Builds a feature vector for an atom.

    :param atom: An RDKit atom.
    :param functional_groups: A k-hot vector indicating the functional groups the atom belongs to.
    :return: A list containing the atom features.
    """
    if atom is None:
        features = [0] * PARAMS.ATOM_FDIM
    else:
        features = onek_encoding_unk(atom.GetAtomicNum() - 1, PARAMS.ATOM_FEATURES['atomic_num']) + \
            onek_encoding_unk(atom.GetTotalDegree(), PARAMS.ATOM_FEATURES['degree']) + \
            onek_encoding_unk(atom.GetFormalCharge(), PARAMS.ATOM_FEATURES['formal_charge']) + \
            onek_encoding_unk(int(atom.GetChiralTag()), PARAMS.ATOM_FEATURES['chiral_tag']) + \
            onek_encoding_unk(int(atom.GetTotalNumHs()), PARAMS.ATOM_FEATURES['num_Hs']) + \
            onek_encoding_unk(int(atom.GetHybridization()), PARAMS.ATOM_FEATURES['hybridization']) + \
            [1 if atom.GetIsAromatic() else 0] + \
            [atom.GetMass() * 0.01]  # scaled to about the same range as other features
        if functional_groups is not None:
            features += functional_groups
    return features

In [8]:
def bond_features(bond: Chem.rdchem.Bond):
    """
    Builds a feature vector for a bond.

    :param bond: An RDKit bond.
    :return: A list containing the bond features.
    """
    if bond is None:
        fbond = [1] + [0] * (PARAMS.BOND_FDIM - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            (bond.GetIsConjugated() if bt is not None else 0),
            (bond.IsInRing() if bt is not None else 0)
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond

In [9]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 2048
#a vector representation (1x2048) for molecular feature 

In [10]:
def morgan_binary_features_generator(mol,
                                     radius: int = MORGAN_RADIUS,
                                     num_bits: int = MORGAN_NUM_BITS):
    """
    Generates a binary Morgan fingerprint for a molecule.
    :param mol: A molecule (i.e., either a SMILES or an RDKit molecule).
    :param radius: Morgan fingerprint radius.
    :param num_bits: Number of bits in Morgan fingerprint.
    :return: A 1D numpy array containing the binary Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features


### Data processing

In [11]:
def data_process(dataset,batch_size):
    SMILES = dataset['SMILES']
    data_list = []
    
    for smiles in SMILES:
            
        mol = Chem.MolFromSmiles(smiles)     
        mol = Chem.AddHs(mol)  

        #generate a global vector features (binary Morgan fingerprint) and convert them
        mol_feature = torch.tensor(morgan_binary_features_generator(mol))

        xs = []
        for atom in mol.GetAtoms():
            x = atom_features(atom)
            xs.append(x)
            
        x = torch.tensor(xs)
        
        edge_indices, edge_attrs = [], []
        
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
    
            e = bond_features(bond)

            edge_indices += [[i,j],[j,i]]
            edge_attrs += [e, e]
        
        edge_index = torch.tensor(edge_indices)
        edge_index = edge_index.t().to(torch.long).view(2, -1)
        edge_attr = torch.tensor(edge_attrs).view(-1, 14)
        
        y = torch.tensor(int(dataset.loc[dataset['SMILES'] == smiles,'Activity'])) #response variable y

        smi = smiles

        # add smiles and num_feature as the attributes
        data = Data(x=x, y=y, edge_index=edge_index, edge_attr = edge_attr, smiles=smi, mol_feature=mol_feature)  
        data_list.append(data)   # store processed data into the list
        
    return DataLoader(data_list,batch_size,shuffle=True)

### Training

In [12]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [13]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [14]:
def train(epoch,train_loader):
    
    model.train()   
    running_loss = 0 
    correct = 0
    total = 0
    criterion = BCELoss()
    for batch in train_loader:

        optimizer.zero_grad()
        outputs = model(batch)
        label = batch.y.view(-1,1)
        loss = criterion(outputs.float(),label.float())
        

        loss.backward()   # Compute the gradient of loss function 
        optimizer.step()  # Update parameters based on gradients.
        running_loss += loss.item()
        # probability that is larger than 0.5, classify as 1 

        pred = (outputs >= 0.5).float()

        total += label.size(0)
        correct += (pred == label).float().sum()
        
    
    loss = running_loss/len(train_loader)
    accuracy = 100*correct/total
    
    train_accuracy.append(accuracy)
    train_loss.append(loss)
    
    if epoch % 10 == 0:
        print('Epoch: '+str(int(epoch)))
        print('Train Loss: %.3f | Accuracy: %.3f'%(loss,accuracy))

In [15]:
def test(epoch,test_loader):
    model.eval()
    
    running_loss = 0
    correct = 0
    total = 0
    
    
    with torch.no_grad():
        criterion = BCELoss()
        for batch in test_loader:
        
            outputs = model(batch)
            label = batch.y.view(-1,1)

            loss = criterion(outputs.float(), label.float())    
            running_loss += loss.item()
            # probability that is larger than 0.5, classify as 1 
            pred = (outputs >= 0.5).float()

            total += label.size(0)
            correct += (pred == label).float().sum()
    
        loss = running_loss/len(test_loader)
        accuracy = 100*correct/total
    
        test_accuracy.append(accuracy)
        test_loss.append(loss)
        if epoch % 10 == 0:
            print('Test Loss: %.3f | Accuracy: %.3f'%(loss,accuracy))

In [16]:
#test_set as a whole loader
def test_metrics(test_loader):
    model.eval()

    with torch.no_grad():
        labels = []
        preds = []
        for batch in test_loader:
            
            labels += list(batch.y.view(-1,1).numpy())
            preds += list(model(batch).detach().numpy())
        
        pred_labels = [1 if i > 0.5 else 0 for i in preds]
        auc = roc_auc_score(list(labels), list(preds), average='weighted')
        report = classification_report(labels, pred_labels,output_dict=True)
        return auc, report
    

In [17]:
def print_metrics(metrics):
    AUC = [] #0
    precision = []
    recall = []
    f1_score = []
    accuracy = []
    for i in metrics:
        AUC.append(i[0])
        precision.append(i[1]['weighted avg']['precision'])
        recall.append(i[1]['weighted avg']['recall'])
        f1_score.append(i[1]['weighted avg']['f1-score'])
        accuracy.append(i[1]['accuracy'])
    
    print('AUC:',np.mean(AUC),'+/-',np.std(AUC))
    print('Accuracy:',np.mean(accuracy),'+/-',np.std(accuracy))
    print('Precision:',np.mean(precision),'+/-',np.std(precision))
    print('Recall:',np.mean(recall),'+/-',np.std(recall))
    print('F1-score:',np.mean(f1_score),'+/-',np.std(f1_score))
    

## FNN analysis

### 1 layer 

### [10]

In [18]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,10)
        self.linear4 = Linear(10, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [19]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    
    


Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=10, bias=True)
Reset trainable parameters of layer = Linear(in_features=10, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.551 | Accuracy: 83.333
Test Loss: 0.479 | Accuracy: 83.333
Validation loss decreased (inf --> 0.478680).  Saving model ...
Validation loss decreased (0.478680 --> 0.397711).  Saving model ...
Validation loss decreased (0.397711 --> 0.352046).  Saving model ...
Validation loss decreased (0.352046 --> 0.316644).  Saving model ...
Validation loss decreased (0.316644 --> 0.291640).  Saving model ...
Validation loss decreased (0.291640 --> 0.276596).  Saving model ...
Validation loss decreased (0.276596 --> 0.263993).  Saving model ...
Validation loss decreased (0.263993 --> 0.257143).  Saving model ...
Validation loss decreased (0.257143 --> 0.2508

EarlyStopping counter: 1 out of 10
Epoch: 20
Train Loss: 0.089 | Accuracy: 98.090
Test Loss: 0.237 | Accuracy: 91.667
Validation loss decreased (0.237799 --> 0.236651).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
Epoch: 30
Train Loss: 0.050 | Accuracy: 99.479
Test Loss: 0.244 | Accuracy: 92.361
EarlyStopping counter: 10 out of 10
Early stopping
Split 5 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=10, bias=True)
Reset trainable parameters of layer = Linear(in_features=10, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.681 | Accuracy: 

In [20]:
print_metrics(metrics)

AUC: 0.9060416666666666 +/- 0.016563045937892658
Accuracy: 0.9194444444444443 +/- 0.007081971546656668
Precision: 0.919746629284548 +/- 0.00906868304630403
Recall: 0.9194444444444443 +/- 0.007081971546656668
F1-score: 0.9116807304424691 +/- 0.008129855558649557


### [250]

In [31]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,250)
        self.linear4 = Linear(250, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [32]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    
    


Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=250, bias=True)
Reset trainable parameters of layer = Linear(in_features=250, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.490 | Accuracy: 80.729
Test Loss: 0.382 | Accuracy: 84.028
Validation loss decreased (inf --> 0.381951).  Saving model ...
Validation loss decreased (0.381951 --> 0.285874).  Saving model ...
Validation loss decreased (0.285874 --> 0.262060).  Saving model ...
Validation loss decreased (0.262060 --> 0.256240).  Saving model ...
Validation loss decreased (0.256240 --> 0.245984).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
Epoch: 10
Train Loss: 0.019 | Accuracy: 100.000
Test Loss: 0.279 | Accuracy:

In [34]:
print_metrics(metrics)    

AUC: 0.9044444444444444 +/- 0.02169224807490478
Accuracy: 0.9125 +/- 0.012108052620946325
Precision: 0.9085729128445598 +/- 0.013253302842692728
Recall: 0.9125 +/- 0.012108052620946325
F1-Score: 0.9068991561466959 +/- 0.013666741020387096


### [1000]

In [35]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,1000)
        self.linear4 = Linear(1000, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [36]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    
    


Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=1000, bias=True)
Reset trainable parameters of layer = Linear(in_features=1000, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.444 | Accuracy: 81.250
Test Loss: 0.323 | Accuracy: 87.500
Validation loss decreased (inf --> 0.322803).  Saving model ...
Validation loss decreased (0.322803 --> 0.282121).  Saving model ...
Validation loss decreased (0.282121 --> 0.274901).  Saving model ...
Validation loss decreased (0.274901 --> 0.259772).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
Epoch: 10
Train Loss: 0.016 | Accuracy: 99.826
Test Loss: 0.344 | Accuracy: 90.278
EarlyStopping counter: 7 

In [38]:
print_metrics(metrics)

AUC: 0.9049305555555556 +/- 0.015418230651399448
Accuracy: 0.9166666666666666 +/- 0.009820927516479838
Precision: 0.9138819055722894 +/- 0.011342375659872922
Recall: 0.9166666666666666 +/- 0.009820927516479838
F1-Score: 0.9107580226840865 +/- 0.011325650887375598


### 2 layers

### [20,10]

In [39]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,20)
        self.linear2 = Linear(20,10)
        self.linear4 = Linear(10, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [40]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=10, bias=True)
Reset trainable parameters of layer = Linear(in_features=10, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.748 | Accuracy: 18.403
Test Loss: 0.698 | Accuracy: 48.611
Validation loss decreased (inf --> 0.697544).  Saving model ...
Validation loss decreased (0.697544 --> 0.545820).  Saving model ...
Validation loss decreased (0.545820 --> 0.432361).  Saving model ...
Validation loss decreased (0.432361 --> 0.397378).  Saving model ...
Validation loss decreased (0.397378 --> 0.361576).  Saving model ...
Validation loss decreased (0.361576 --> 0.333137).  Saving model ...
Validation loss decreased (0.333137 --> 0.312266).  Saving model ...
Validation loss decreased 

Validation loss decreased (0.247548 --> 0.244755).  Saving model ...
Epoch: 10
Train Loss: 0.077 | Accuracy: 98.264
Test Loss: 0.245 | Accuracy: 92.361
Validation loss decreased (0.244755 --> 0.244639).  Saving model ...
Validation loss decreased (0.244639 --> 0.244485).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
Epoch: 20
Train Loss: 0.019 | Accuracy: 99.826
Test Loss: 0.275 | Accuracy: 91.667
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping


In [42]:
print_metrics(metrics)

AUC: 0.9060416666666666 +/- 0.014508072997430276
Accuracy: 0.913888888888889 +/- 0.007081971546656651
Precision: 0.9116947760373536 +/- 0.006449467528059945
Recall: 0.913888888888889 +/- 0.007081971546656651
F1-Score: 0.9059422184139564 +/- 0.009749964326008685


### [500,250]

In [43]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,500)
        self.linear2 = Linear(500,250)
        self.linear4 = Linear(250, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [44]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=500, bias=True)
Reset trainable parameters of layer = Linear(in_features=500, out_features=250, bias=True)
Reset trainable parameters of layer = Linear(in_features=250, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.481 | Accuracy: 80.556
Test Loss: 0.351 | Accuracy: 84.722
Validation loss decreased (inf --> 0.351207).  Saving model ...
Validation loss decreased (0.351207 --> 0.269777).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
Epoch: 10
Train Loss: 0.003 | Accuracy: 100.000
Test Loss: 0.443 | Accuracy: 90.972
Ear

In [46]:
print_metrics(metrics)

AUC: 0.8920833333333335 +/- 0.01998673364021589
Accuracy: 0.9180555555555555 +/- 0.002777777777777812
Precision: 0.916693519944759 +/- 0.0029809777101818514
Recall: 0.9180555555555555 +/- 0.002777777777777812
F1-Score: 0.9112297302994973 +/- 0.004144164460429964


### [1000, 750]

In [50]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,1000)
        self.linear2 = Linear(1000,750)
        self.linear4 = Linear(750, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [51]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=1000, bias=True)
Reset trainable parameters of layer = Linear(in_features=1000, out_features=750, bias=True)
Reset trainable parameters of layer = Linear(in_features=750, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.457 | Accuracy: 80.035
Test Loss: 0.325 | Accuracy: 86.111
Validation loss decreased (inf --> 0.324861).  Saving model ...
Validation loss decreased (0.324861 --> 0.269370).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
Epoch: 10
Train Loss: 0.004 | Accuracy: 100.000
Test Loss: 0.395 | Accuracy: 90.278
E

In [53]:
print_metrics(metrics)

AUC: 0.900625 +/- 0.018285659816869398
Accuracy: 0.9125 +/- 0.012880025688188482
Precision: 0.9082433779468768 +/- 0.01435870230375002
Recall: 0.9125 +/- 0.012880025688188482
F1-Score: 0.9079363730714591 +/- 0.013249202856802578


### 3 Layers

### [50,20,10]

In [58]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,50)
        self.linear2 = Linear(50,20)
        self.linear3 = Linear(20,10)
        self.linear4 = Linear(10, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [59]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=10, bias=True)
Reset trainable parameters of layer = Linear(in_features=10, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.640 | Accuracy: 83.333
Test Loss: 0.596 | Accuracy: 83.333
Validation loss decreased (inf --> 0.595925).  Saving model ...
Validation loss decreased (0.595925 --> 0.442704).  Saving model ...
Validation loss decreased (0.442704 --> 0.370737).  Saving model ...
Validation loss decreased (0.370737 --> 0.328548).  Saving model ...
Validation loss decreased (0.328548 --> 0.299439).  Saving model ...
Validation loss decreased (0.299439 --> 0.283510).  Saving model ...
Valida

In [61]:
print_metrics(metrics)

AUC: 0.90625 +/- 0.013093546968725554
Accuracy: 0.9180555555555555 +/- 0.01020620726159659
Precision: 0.9162778153204384 +/- 0.01180417318758369
Recall: 0.9180555555555555 +/- 0.01020620726159659
F1-Score: 0.9108043659636218 +/- 0.011957614768403063


### [500,250,10]

In [64]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,500)
        self.linear2 = Linear(500,250)
        self.linear3 = Linear(250,100)
        self.linear4 = Linear(100, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [65]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=500, bias=True)
Reset trainable parameters of layer = Linear(in_features=500, out_features=250, bias=True)
Reset trainable parameters of layer = Linear(in_features=250, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=100, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.518 | Accuracy: 83.333
Test Loss: 0.374 | Accuracy: 83.333
Validation loss decreased (inf --> 0.374346).  Saving model ...
Validation loss decreased (0.374346 --> 0.299280).  Saving model ...
Validation loss decreased (0.299280 --> 0.267340).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 

In [67]:
print_metrics(metrics)

AUC: 0.8897916666666668 +/- 0.02141727913809341
Accuracy: 0.913888888888889 +/- 0.008333333333333356
Precision: 0.9122939351627937 +/- 0.006989355462418052
Recall: 0.913888888888889 +/- 0.008333333333333356
F1-Score: 0.9088619611916652 +/- 0.008436960853723306


### [1000,750,500]

In [68]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,1000)
        self.linear2 = Linear(1000,750)
        self.linear3 = Linear(750,500)
        self.linear4 = Linear(500, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

In [69]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('C:/Users/jimmy/Desktop/FYP/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    

Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=1000, bias=True)
Reset trainable parameters of layer = Linear(in_features=1000, out_features=750, bias=True)
Reset trainable parameters of layer = Linear(in_features=750, out_features=500, bias=True)
Reset trainable parameters of layer = Linear(in_features=500, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.423 | Accuracy: 83.507
Test Loss: 0.340 | Accuracy: 86.806
Validation loss decreased (inf --> 0.339770).  Saving model ...
Validation loss decreased (0.339770 --> 0.290800).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out 

In [71]:
print_metrics(metrics)

AUC: 0.8995833333333334 +/- 0.01902118701752223
Accuracy: 0.9138888888888888 +/- 0.007081971546656651
Precision: 0.9100674017550077 +/- 0.007280782787461408
Recall: 0.9138888888888888 +/- 0.007081971546656651
F1-Score: 0.9088795223324763 +/- 0.009652213392634194
